### 코드북 맵핑
1. 불필요한 컬럼 제거 (종속변수 파생변수 생성 전의 원자료 컬럼, pid 인덱스)
2. 데이터 구조 점검
3. 기존 코드북 맵핑
4. 일자리에 대한 주관적 기대감 파생변수로 통합

- 최종적으로 각 코드들을 실제 범주 데이터로 맵핑하고
- null 값 조사 구조 기반으로 전처리 하고
- 범주형 데이터 category 형으로 변경 완료
- w09g003~w09g004b(연령대별 주관적 노동가능 연령 인식) 변수는 응답자 연령대에 따라 하나만 값이 존재하므로 COALESCE 방식으로 work_ability_age 단일 변수로 통합함. 해당 변수의 NaN은 현재 비근로 상태를 의미함. 원본 4개 변수는 삭제함.

#### 라이브러리 불러오기

In [48]:
from pandas import DataFrame, read_csv, read_excel
import pandas as pd

In [49]:
making_derived_variable_completed = read_csv(r'C:\itwill_bigdata_final_project-main\itwill_bigdata_final_project\1. 초기 데이터 전처리\2.making_derived_variable_completed.csv',encoding='utf-8')


making_derived_variable_completed.head()

,pid,w09_fam1,w09_fam2,w09hhsize,w09edu,w09gender1,w09a002_age,w09marital,w09year2,w09edu_s,w09ecoact_s,w09enu_type,w09ba003,w09livewithnm,w09ba069,w09bp1,w09c152,w09c001,w09c003,w09c005,w09chronic_a,w09chronic_b,w09chronic_c,w09chronic_d,w09chronic_e,w09chronic_f,w09chronic_g,w09chronic_h,w09chronic_i,w09chronic_j,w09chronic_k,w09chronic_l,w09chronic_m,w09c056,w09c068,w09c081,w09c082,w09c085,w09c102,w09bmi,w09c108,w09c111,w09smoke,w09alc,w09addic,w09c318,w09c330,w09c333,w09c334,w09c337,w09c340,w09adl,w09c550,w09mgrip,w09fromchildren,w09tochildren,w09transferfrom,w09transferto,w09earned,w09pinc,w09e201,w09e207,w09e213,w09e219,w09e225,w09e231,w09e237,w09e243,w09e273,w09e251,w09passets,w09pliabilities,w09pnetassets,w09f001type,w09hhinc,w09hhassets,w09hhliabilities,w09hhnetassets,w09g032,w09g003,w09g004,w09g004a,w09g004b,w09g026,w09g027,w09g028,w09g029,w09g030,w09g031,w09cadd_19,w09c142,w09c143,w09c144,w09c145,w09c146,w09c147,w09c148,w09c149,w09c150,w09c151,w09present_ecotype,w09d001,w09d002,dependent_ecotype,dependent_wage_work
0,21,1,2,2,4,5,67,1,1954,4,1,1,2,NaN,5.000,4,3,3,5.000,3,5,1,5,5,5,5,5,5,1,NaN,5,5,NaN,5,NaN,5.000,5,5,5.000,24.030,1,5.000,0,3,NaN,0,0,0,0,2,0,0,1,21.500,15.000,NaN,15.000,NaN,NaN,375.000,-9.000,-9.000,-9.000,-9.000,-9.000,-9.000,-9.000,-9.000,-9.000,-9.000,35500.000,NaN,35500.000,2,13000,30000,10000,20000,20,NaN,NaN,NaN,NaN,70,60,70.000,70.000,80,3,5,1,1,1,1,3,1,1,1,4,1,3,5,NaN,0,0
1,22,1,2,2,4,1,68,1,1955,4,3,1,2,NaN,5.000,4,3,3,5.000,2,5,5,5,5,5,5,5,5,5,1.000,5,5,NaN,5,NaN,5.000,5,5,5.000,21.470,1,5.000,1,1,0.000,0,0,0,0,4,0,0,5,NaN,15.000,NaN,15.000,NaN,2400.000,2415.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,13000,30000,10000,20000,20,NaN,NaN,NaN,30.000,70,70,70.000,NaN,60,3,5,1,1,1,1,4,1,1,3,1,1,1,1,1.000,1,1
2,51,1,2,2,1,1,80,1,1946,1,3,1,2,NaN,5.000,4,2,2,5.000,3,1,1,5,5,5,5,5,5,1,5.000,5,5,NaN,5,NaN,5.000,5,5,5.000,21.970,1,7.000,1,2,NaN,0,0,0,0,6,0,0,1,31.500,175.000,50.000,175.000,50.000,NaN,10663.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.000,10000.000,20000.000,1,11200,33000,10000,23000,10,NaN,NaN,NaN,NaN,70,60,40.000,NaN,60,4,5,1,2,2,2,3,2,1,3,2,1,3,5,NaN,0,0
3,52,1,2,2,1,5,76,1,1942,1,3,1,2,NaN,5.000,4,3,3,5.000,3,1,1,5,5,5,5,5,5,1,NaN,5,5,NaN,5,5.000,5.000,5,1,1.000,21.480,1,5.000,0,2,NaN,0,0,0,0,6,0,0,1,19.250,175.000,50.000,175.000,50.000,NaN,823.000,100.000,50.000,5.000,0.000,0.000,13.000,15.000,5.000,0.000,0.000,10200.000,10000.000,200.000,1,11200,33000,10000,23000,20,NaN,NaN,NaN,NaN,70,60,30.000,80.000,60,4,5,1,2,1,2,3,1,1,3,2,1,3,5,NaN,0,0
4,62,1,1,1,1,5,78,4,-6,-6,-6,1,3,NaN,5.000,4,3,3,5.000,4,5,5,5,5,5,5,5,5,5,NaN,5,5,NaN,5,NaN,5.000,5,1,1.000,22.010,1,5.000,0,3,NaN,0,0,0,0,6,0,0,1,17.000,170.000,NaN,170.000,NaN,NaN,710.000,100.000,40.000,5.000,0.000,0.000,10.000,5.000,5.000,5.000,20.000,15000.000,10000.000,5000.000,1,11300,16000,10000,6000,10,NaN,NaN,NaN,NaN,60,60,NaN,70.000,70,5,5,1,2,2,1,3,2,3,3,2,2,3,5,NaN,0,0


#### 종속변수 DROP
<DROP 목록>
- w09present_ecotype
- w09d001
- w09d002

In [50]:
making_derived_variable_completed_drop= making_derived_variable_completed.copy()
making_derived_variable_completed_drop=making_derived_variable_completed_drop.drop(['w09present_ecotype','w09d001','w09d002','pid'],axis=1)

making_derived_variable_completed_drop.head()


,w09_fam1,w09_fam2,w09hhsize,w09edu,w09gender1,w09a002_age,w09marital,w09year2,w09edu_s,w09ecoact_s,w09enu_type,w09ba003,w09livewithnm,w09ba069,w09bp1,w09c152,w09c001,w09c003,w09c005,w09chronic_a,w09chronic_b,w09chronic_c,w09chronic_d,w09chronic_e,w09chronic_f,w09chronic_g,w09chronic_h,w09chronic_i,w09chronic_j,w09chronic_k,w09chronic_l,w09chronic_m,w09c056,w09c068,w09c081,w09c082,w09c085,w09c102,w09bmi,w09c108,w09c111,w09smoke,w09alc,w09addic,w09c318,w09c330,w09c333,w09c334,w09c337,w09c340,w09adl,w09c550,w09mgrip,w09fromchildren,w09tochildren,w09transferfrom,w09transferto,w09earned,w09pinc,w09e201,w09e207,w09e213,w09e219,w09e225,w09e231,w09e237,w09e243,w09e273,w09e251,w09passets,w09pliabilities,w09pnetassets,w09f001type,w09hhinc,w09hhassets,w09hhliabilities,w09hhnetassets,w09g032,w09g003,w09g004,w09g004a,w09g004b,w09g026,w09g027,w09g028,w09g029,w09g030,w09g031,w09cadd_19,w09c142,w09c143,w09c144,w09c145,w09c146,w09c147,w09c148,w09c149,w09c150,w09c151,dependent_ecotype,dependent_wage_work
0,1,2,2,4,5,67,1,1954,4,1,1,2,NaN,5.000,4,3,3,5.000,3,5,1,5,5,5,5,5,5,1,NaN,5,5,NaN,5,NaN,5.000,5,5,5.000,24.030,1,5.000,0,3,NaN,0,0,0,0,2,0,0,1,21.500,15.000,NaN,15.000,NaN,NaN,375.000,-9.000,-9.000,-9.000,-9.000,-9.000,-9.000,-9.000,-9.000,-9.000,-9.000,35500.000,NaN,35500.000,2,13000,30000,10000,20000,20,NaN,NaN,NaN,NaN,70,60,70.000,70.000,80,3,5,1,1,1,1,3,1,1,1,4,1,0,0
1,1,2,2,4,1,68,1,1955,4,3,1,2,NaN,5.000,4,3,3,5.000,2,5,5,5,5,5,5,5,5,5,1.000,5,5,NaN,5,NaN,5.000,5,5,5.000,21.470,1,5.000,1,1,0.000,0,0,0,0,4,0,0,5,NaN,15.000,NaN,15.000,NaN,2400.000,2415.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,13000,30000,10000,20000,20,NaN,NaN,NaN,30.000,70,70,70.000,NaN,60,3,5,1,1,1,1,4,1,1,3,1,1,1,1
2,1,2,2,1,1,80,1,1946,1,3,1,2,NaN,5.000,4,2,2,5.000,3,1,1,5,5,5,5,5,5,1,5.000,5,5,NaN,5,NaN,5.000,5,5,5.000,21.970,1,7.000,1,2,NaN,0,0,0,0,6,0,0,1,31.500,175.000,50.000,175.000,50.000,NaN,10663.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.000,10000.000,20000.000,1,11200,33000,10000,23000,10,NaN,NaN,NaN,NaN,70,60,40.000,NaN,60,4,5,1,2,2,2,3,2,1,3,2,1,0,0
3,1,2,2,1,5,76,1,1942,1,3,1,2,NaN,5.000,4,3,3,5.000,3,1,1,5,5,5,5,5,5,1,NaN,5,5,NaN,5,5.000,5.000,5,1,1.000,21.480,1,5.000,0,2,NaN,0,0,0,0,6,0,0,1,19.250,175.000,50.000,175.000,50.000,NaN,823.000,100.000,50.000,5.000,0.000,0.000,13.000,15.000,5.000,0.000,0.000,10200.000,10000.000,200.000,1,11200,33000,10000,23000,20,NaN,NaN,NaN,NaN,70,60,30.000,80.000,60,4,5,1,2,1,2,3,1,1,3,2,1,0,0
4,1,1,1,1,5,78,4,-6,-6,-6,1,3,NaN,5.000,4,3,3,5.000,4,5,5,5,5,5,5,5,5,5,NaN,5,5,NaN,5,NaN,5.000,5,1,1.000,22.010,1,5.000,0,3,NaN,0,0,0,0,6,0,0,1,17.000,170.000,NaN,170.000,NaN,NaN,710.000,100.000,40.000,5.000,0.000,0.000,10.000,5.000,5.000,5.000,20.000,15000.000,10000.000,5000.000,1,11300,16000,10000,6000,10,NaN,NaN,NaN,NaN,60,60,NaN,70.000,70,5,5,1,2,2,1,3,2,3,3,2,2,0,0


### 데이터 구조 점검

In [51]:
origin= making_derived_variable_completed_drop.copy()

In [52]:
rows,cols = origin.shape
print(f"{rows} 행, {cols} 열의 {origin.ndim} 차원 데이터")

6057 행, 101 열의 2 차원 데이터


#### 전체데이터 구조 점검

In [53]:
dtype_df = pd.DataFrame(origin.dtypes, columns=['dtype'])
dtype_df.reset_index(inplace=True)
dtype_df.columns = ['column_name', 'dtype']
pd.set_option('display.max_rows', None)

dtype_df


,column_name,dtype
0,w09_fam1,int64
1,w09_fam2,int64
2,w09hhsize,int64
3,w09edu,int64
4,w09gender1,int64
5,w09a002_age,int64
6,w09marital,int64
7,w09year2,int64
8,w09edu_s,int64
9,w09ecoact_s,int64


#### 명목형 데이터 구조/구성 점검

In [54]:
#명목형 데이터의 구성 확인
import pandas as pd
import numpy as np
from IPython.display import display

# =========================
# 0) (선택) 출력 옵션: 생략 없이
# =========================
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# =========================
# 1) 지정한 "명목형" 컬럼 리스트
# =========================
categorical_cols = [
    'w09_fam1','w09_fam2','w09edu','w09gender1','w09marital','w09edu_s','w09ecoact_s','w09enu_type',
    'w09ba069','w09bp1','w09c152','w09c001','w09c003','w09c005',
    'w09chronic_a','w09chronic_b','w09chronic_c','w09chronic_d','w09chronic_e','w09chronic_f',
    'w09chronic_g','w09chronic_h','w09chronic_i','w09chronic_j','w09chronic_k','w09chronic_l','w09chronic_m',
    'w09c056','w09c068','w09c081','w09c082','w09c085','w09c102',
    'w09smoke','w09alc','w09addic','w09c550',
    'w09f001type','w09g031',
    'w09cadd_19','w09c142','w09c143','w09c144','w09c145','w09c146','w09c147','w09c148','w09c149','w09c150','w09c151',
    'dependent_ecotype','dependent_wage_work'
]


# =========================
# 2) 실제 데이터에 없는 컬럼 자동 제거 (안전장치)
# =========================
missing_cols = [c for c in categorical_cols if c not in origin.columns]
if missing_cols:
    print("⚠️ 데이터에 없는 명목형 컬럼(자동 제외):", missing_cols)

categorical_cols = [c for c in categorical_cols if c in origin.columns]

print(f"✅ 최종 명목형 컬럼 수: {len(categorical_cols)}")

# =========================
# 3) 명목형 컬럼들 dtype 한 번에 확인(표)
# =========================
cat_dtype_df = origin[categorical_cols].dtypes.to_frame('dtype').reset_index()
cat_dtype_df.columns = ['column_name', 'dtype']
display(cat_dtype_df)

# =========================
# 4) 명목형 컬럼들 값 구성(value_counts) 정리해서 표로 만들기
#    - 코드값 기준 정렬(sort_index)
# =========================
summary_list = []
for col in categorical_cols:
    vc = origin[col].value_counts(dropna=False).sort_index()
    tmp = vc.reset_index()
    tmp.columns = ['value', 'count']
    tmp['column'] = col
    summary_list.append(tmp)

cat_summary_df = pd.concat(summary_list, ignore_index=True)
cat_summary_df = cat_summary_df[['column', 'value', 'count']]

display(cat_summary_df)





✅ 최종 명목형 컬럼 수: 52


,column_name,dtype
0,w09_fam1,int64
1,w09_fam2,int64
2,w09edu,int64
3,w09gender1,int64
4,w09marital,int64
5,w09edu_s,int64
6,w09ecoact_s,int64
7,w09enu_type,int64
8,w09ba069,float64
9,w09bp1,int64


,column,value,count
0,w09_fam1,1.000,4212
1,w09_fam1,2.000,1592
2,w09_fam1,3.000,238
3,w09_fam1,4.000,14
4,w09_fam1,5.000,1
5,w09_fam2,1.000,1138
6,w09_fam2,2.000,3057
7,w09_fam2,3.000,17
8,w09_fam2,4.000,15
9,w09_fam2,5.000,1575


In [55]:
#각 변수의 데이터 타입 확인
origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6057 entries, 0 to 6056
Columns: 101 entries, w09_fam1 to dependent_wage_work
dtypes: float64(37), int64(64)
memory usage: 4.7 MB


#### 명목형 변수 처리
- 예/아니오 응답 > 예 : 1 , 아니오 0
- 그 외 3개 이상 : 각 범주의 텍스트 이름으로 숫자값 대체! 

In [56]:
# =========================
# 0) 공통 결측코드 -> NaN
# =========================
NULL_CODES = [-6, -8, -9, 97]

# =========================
# 1) 명목형 매핑 사전(누락 없이)
# =========================
mapping_dict = {
    # 가구/개인 특성
    "w09_fam1": {
        1: "1세대",
        2: "2세대",
        3: "3세대",
        4: "4세대",
        5: "기타",
    },
    "w09_fam2": {
        1: "단독가구",
        2: "부부",
        3: "조손",
        4: "1세대",
        5: "2세대",
        6: "3세대",
        7: "4세대",
        8: "기타",
    },
    "w09edu": {
        1: "초졸이하",
        2: "중졸",
        3: "고졸",
        4: "대졸이상",
    },
    "w09gender1": {
        1: "남",
        5: "여",
    },
    "w09marital": {
        1: "혼인중",
        2: "별거",
        3: "이혼",
        4: "사별 또는 실종 (이산가족)",
        5: "결혼한 적 없음",
    },
    "w09edu_s": {
        1: "초졸이하",
        2: "중졸",
        3: "고졸",
        4: "대졸이상",
    },
    "w09ecoact_s": {
        1: "취업",
        2: "구직",
        3: "비경활",
    },

    #추후 확인 필요한 변수
    "w09enu_type": {
        1: "1",
        2: "2",
    },
    "w09ba069": {
        1: "예",
        5: "아니오",
    },
    "w09bp1": {
        1: "두분 모두",
        2: "아버님",
        3: "어머님",
        4: "두분 모두 사망 혹은 안 계심 (실종)",
    },

    # 건강/상태(서열 포함)
    "w09c152": {
        1: "매우 좋음",
        2: "좋은 편",
        3: "보통",
        4: "나쁜 편",
        5: "매우 나쁨",
    },
    "w09c001": {
        1: "최상",
        2: "매우 좋음",
        3: "좋은 편",
        4: "보통",
        5: "나쁜 편",
    },
    "w09c003": {
        1: "예",
        5: "아니오",
    },
    "w09c005": {
        1: "매우 그렇다",
        2: "그런 편이다",
        3: "그렇지 않은 편이다",
        4: "전혀 그렇지 않다",
    },

    # 만성질환/예아니오(코딩북 기준 1=예, 5=아니오)
    "w09chronic_a": {1: "예", 5: "아니오"},
    "w09chronic_b": {1: "예", 5: "아니오"},
    "w09chronic_c": {1: "예", 5: "아니오"},
    "w09chronic_d": {1: "예", 5: "아니오"},
    "w09chronic_e": {1: "예", 5: "아니오"},
    "w09chronic_f": {1: "예", 5: "아니오"},
    "w09chronic_g": {1: "예", 5: "아니오"},
    "w09chronic_h": {1: "예", 5: "아니오"},
    "w09chronic_i": {1: "예", 5: "아니오"},
    "w09chronic_j": {1: "예", 5: "아니오"},
    "w09chronic_k": {1: "예", 5: "아니오"},
    "w09chronic_l": {1: "예", 5: "아니오"},
    "w09chronic_m": {1: "예", 5: "아니오"},
    "w09c056":      {1: "예", 5: "아니오"},
    "w09c068":      {1: "예", 5: "아니오"},
    "w09c081":      {1: "예", 5: "아니오"},
    "w09c082":      {1: "예", 5: "아니오"},
    "w09c085":      {1: "예", 5: "아니오"},
    "w09c102":      {1: "예", 5: "아니오"},
    "w09c550":      {1: "예", 5: "아니오"},

    # 흡연/음주
    "w09smoke": {
        0: "비흡연자",
        1: "과거흡연자",
        2: "현재흡연자",
    },
    "w09alc": {
        1: "현재음주자",
        2: "과거음주자",
        3: "비음주자",
        # 코딩북에서 10에 '결측' 표시가 있었는데,
        # 우리는 결측코드(-6,-8,-9,97)만 NaN 처리하므로 10은 매핑하지 않음(남겨두기)
    },
    "w09addic": {
        0: "정상",
        1: "과음",
        2: "알코올 중독 수준",
    },

    # 주거형태
    "w09f001type": {
        1: "자기집",
        2: "전세",
        3: "보증급 있는 월세",
        4: "보증금 없는 월세",
        5: "기타",
    },

    # 주관적 계층의식
    "w09g031": {
        1: "상의 상",
        2: "상의 하",
        3: "중의 상",
        4: "중의 하",
        5: "하의 상",
        6: "하의 하",
    },

    # 우울/정신건강(특수)
    "w09cadd_19": {
        1: "예",
        3: "항우울제복용",
        5: "아니오",
    },

    # CES-D10 항목(지난 1주일 빈도)
    "w09c142": {1: "하루 미만", 2: "하루 이틀", 3: "3-4일", 4: "5-7일"},
    "w09c143": {1: "하루 미만", 2: "하루 이틀", 3: "3-4일", 4: "5-7일"},
    "w09c144": {1: "하루 미만", 2: "하루 이틀", 3: "3-4일", 4: "5-7일"},
    "w09c145": {1: "하루 미만", 2: "하루 이틀", 3: "3-4일", 4: "5-7일"},
    "w09c146": {1: "하루 미만", 2: "하루 이틀", 3: "3-4일", 4: "5-7일"},
    "w09c147": {1: "하루 미만", 2: "하루 이틀", 3: "3-4일", 4: "5-7일"},
    "w09c148": {1: "하루 미만", 2: "하루 이틀", 3: "3-4일", 4: "5-7일"},
    "w09c149": {1: "하루 미만", 2: "하루 이틀", 3: "3-4일", 4: "5-7일"},
    "w09c150": {1: "하루 미만", 2: "하루 이틀", 3: "3-4일", 4: "5-7일"},
    "w09c151": {1: "하루 미만", 2: "하루 이틀", 3: "3-4일", 4: "5-7일"},


    # 경제활동 상태(현재 경활)
    "w09present_ecotype": {
        1: "취업자",
        2: "실업자",
        3: "비경활",
    },

    # -------------------------
    # 종속변수
    # -------------------------
    "dependent_ecotype": {
        0: 0,
        1: 1,
    },

    "dependent_wage_work": {
        0: 0,
        1: 1,
    }

}





In [57]:
import numpy as np

# 1️⃣ 결측코드 처리
NULL_CODES = [-6, -8, -9, 97]
origin2 = origin.copy()
origin2 = origin2.replace(NULL_CODES, np.nan)

# 2️⃣ 명목형 컬럼을 "그 자리에서 바로" 라벨로 변환
for col, mp in mapping_dict.items():
    if col in origin2.columns:
        origin2[col] = origin2[col].map(mp)

# 3️⃣ 확인
origin2.head()


,w09_fam1,w09_fam2,w09hhsize,w09edu,w09gender1,w09a002_age,w09marital,w09year2,w09edu_s,w09ecoact_s,w09enu_type,w09ba003,w09livewithnm,w09ba069,w09bp1,w09c152,w09c001,w09c003,w09c005,w09chronic_a,w09chronic_b,w09chronic_c,w09chronic_d,w09chronic_e,w09chronic_f,w09chronic_g,w09chronic_h,w09chronic_i,w09chronic_j,w09chronic_k,w09chronic_l,w09chronic_m,w09c056,w09c068,w09c081,w09c082,w09c085,w09c102,w09bmi,w09c108,w09c111,w09smoke,w09alc,w09addic,w09c318,w09c330,w09c333,w09c334,w09c337,w09c340,w09adl,w09c550,w09mgrip,w09fromchildren,w09tochildren,w09transferfrom,w09transferto,w09earned,w09pinc,w09e201,w09e207,w09e213,w09e219,w09e225,w09e231,w09e237,w09e243,w09e273,w09e251,w09passets,w09pliabilities,w09pnetassets,w09f001type,w09hhinc,w09hhassets,w09hhliabilities,w09hhnetassets,w09g032,w09g003,w09g004,w09g004a,w09g004b,w09g026,w09g027,w09g028,w09g029,w09g030,w09g031,w09cadd_19,w09c142,w09c143,w09c144,w09c145,w09c146,w09c147,w09c148,w09c149,w09c150,w09c151,dependent_ecotype,dependent_wage_work
0,1세대,부부,2,대졸이상,여,67.000,혼인중,1954.000,대졸이상,취업,1,2,NaN,아니오,두분 모두 사망 혹은 안 계심 (실종),보통,좋은 편,아니오,그렇지 않은 편이다,아니오,예,아니오,아니오,아니오,아니오,아니오,아니오,예,NaN,아니오,아니오,NaN,아니오,NaN,아니오,아니오,아니오,아니오,24.030,1,5.000,비흡연자,비음주자,NaN,0.000,0.000,0.000,0.000,2.000,0.000,0,예,21.500,15.000,NaN,15.000,NaN,NaN,375.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35500.000,NaN,35500.000,전세,13000.000,30000.000,10000.000,20000.000,20.000,NaN,NaN,NaN,NaN,70.000,60.000,70.000,70.000,80.000,중의 상,아니오,하루 미만,하루 미만,하루 미만,하루 미만,3-4일,하루 미만,하루 미만,하루 미만,5-7일,하루 미만,0,0
1,1세대,부부,2,대졸이상,남,68.000,혼인중,1955.000,대졸이상,비경활,1,2,NaN,아니오,두분 모두 사망 혹은 안 계심 (실종),보통,좋은 편,아니오,그런 편이다,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,예,아니오,아니오,NaN,아니오,NaN,아니오,아니오,아니오,아니오,21.470,1,5.000,과거흡연자,현재음주자,정상,0.000,0.000,0.000,0.000,4.000,0.000,0,아니오,NaN,15.000,NaN,15.000,NaN,2400.000,2415.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,전세,13000.000,30000.000,10000.000,20000.000,20.000,NaN,NaN,NaN,30.000,70.000,70.000,70.000,NaN,60.000,중의 상,아니오,하루 미만,하루 미만,하루 미만,하루 미만,5-7일,하루 미만,하루 미만,3-4일,하루 미만,하루 미만,1,1
2,1세대,부부,2,초졸이하,남,80.000,혼인중,1946.000,초졸이하,비경활,1,2,NaN,아니오,두분 모두 사망 혹은 안 계심 (실종),좋은 편,매우 좋음,아니오,그렇지 않은 편이다,예,예,아니오,아니오,아니오,아니오,아니오,아니오,예,아니오,아니오,아니오,NaN,아니오,NaN,아니오,아니오,아니오,아니오,21.970,1,7.000,과거흡연자,과거음주자,NaN,0.000,0.000,0.000,0.000,6.000,0.000,0,예,31.500,175.000,50.000,175.000,50.000,NaN,10663.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.000,10000.000,20000.000,자기집,11200.000,33000.000,10000.000,23000.000,10.000,NaN,NaN,NaN,NaN,70.000,60.000,40.000,NaN,60.000,중의 하,아니오,하루 미만,하루 이틀,하루 이틀,하루 이틀,3-4일,하루 이틀,하루 미만,3-4일,하루 이틀,하루 미만,0,0
3,1세대,부부,2,초졸이하,여,76.000,혼인중,1942.000,초졸이하,비경활,1,2,NaN,아니오,두분 모두 사망 혹은 안 계심 (실종),보통,좋은 편,아니오,그렇지 않은 편이다,예,예,아니오,아니오,아니오,아니오,아니오,아니오,예,NaN,아니오,아니오,NaN,아니오,아니오,아니오,아니오,예,예,21.480,1,5.000,비흡연자,과거음주자,NaN,0.000,0.000,0.000,0.000,6.000,0.000,0,예,19.250,175.000,50.000,175.000,50.000,NaN,823.000,100.000,50.000,5.000,0.000,0.000,13.000,15.000,5.000,0.000,0.000,10200.000,10000.000,200.000,자기집,11200.000,33000.000,10000.000,23000.000,20.000,NaN,NaN,NaN,NaN,70.000,60.000,30.000,80.000,60.000,중의 하,아니오,하루 미만,하루 이틀,하루 미만,하루 이틀,3-4일,하루 미만,하루 미만,3-4일,하루 이틀,하루 미만,0,0
4,1세대,단독가구,1,초졸이하,여,78.000,사별 또는 실종 (이산가족),NaN,NaN,NaN,1,3,NaN,아니오,두분 모두 사망 혹은 안 계심 (실종),보통,좋은 편,아니오,전혀 그렇지 않다,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,NaN,아니오,아니오,NaN,아니오,NaN,아니오,아니오,예,예,22.010,1,5.000,비흡연자,비음주자,NaN,0.000,0.000,0.000,0.000,6.000,0.000,0,예,17.000,170.000,NaN,170.000,NaN,NaN,710.000,100.000,40.000,5.000,0.000,0.000,10.000,5.000,5.000,5.000,20.000,15000.000,10000.000,5000.000,자기집,11300.000,16000.000,10000.000,6000.000,10.000,NaN,NaN,NaN,NaN,60.000,60.000,NaN,70.000,70.000,하의 상,아니오,하루 미만,하루 이틀,하루 이틀,하루 미만,3-4일,하루 이틀,3-4일,3-4일,하루 이틀,하루 이틀,0,0


### 추가 null 처리

In [58]:
# w09ba069: NaN -> "손자녀 없음"
origin2["w09ba069"] = origin2["w09ba069"].fillna("손자녀 없음")



origin2["w09c003"] = origin2["w09c003"].fillna("이전 조사에서 이미 장애판정")




In [59]:


origin2.head()

,w09_fam1,w09_fam2,w09hhsize,w09edu,w09gender1,w09a002_age,w09marital,w09year2,w09edu_s,w09ecoact_s,w09enu_type,w09ba003,w09livewithnm,w09ba069,w09bp1,w09c152,w09c001,w09c003,w09c005,w09chronic_a,w09chronic_b,w09chronic_c,w09chronic_d,w09chronic_e,w09chronic_f,w09chronic_g,w09chronic_h,w09chronic_i,w09chronic_j,w09chronic_k,w09chronic_l,w09chronic_m,w09c056,w09c068,w09c081,w09c082,w09c085,w09c102,w09bmi,w09c108,w09c111,w09smoke,w09alc,w09addic,w09c318,w09c330,w09c333,w09c334,w09c337,w09c340,w09adl,w09c550,w09mgrip,w09fromchildren,w09tochildren,w09transferfrom,w09transferto,w09earned,w09pinc,w09e201,w09e207,w09e213,w09e219,w09e225,w09e231,w09e237,w09e243,w09e273,w09e251,w09passets,w09pliabilities,w09pnetassets,w09f001type,w09hhinc,w09hhassets,w09hhliabilities,w09hhnetassets,w09g032,w09g003,w09g004,w09g004a,w09g004b,w09g026,w09g027,w09g028,w09g029,w09g030,w09g031,w09cadd_19,w09c142,w09c143,w09c144,w09c145,w09c146,w09c147,w09c148,w09c149,w09c150,w09c151,dependent_ecotype,dependent_wage_work
0,1세대,부부,2,대졸이상,여,67.000,혼인중,1954.000,대졸이상,취업,1,2,NaN,아니오,두분 모두 사망 혹은 안 계심 (실종),보통,좋은 편,아니오,그렇지 않은 편이다,아니오,예,아니오,아니오,아니오,아니오,아니오,아니오,예,NaN,아니오,아니오,NaN,아니오,NaN,아니오,아니오,아니오,아니오,24.030,1,5.000,비흡연자,비음주자,NaN,0.000,0.000,0.000,0.000,2.000,0.000,0,예,21.500,15.000,NaN,15.000,NaN,NaN,375.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35500.000,NaN,35500.000,전세,13000.000,30000.000,10000.000,20000.000,20.000,NaN,NaN,NaN,NaN,70.000,60.000,70.000,70.000,80.000,중의 상,아니오,하루 미만,하루 미만,하루 미만,하루 미만,3-4일,하루 미만,하루 미만,하루 미만,5-7일,하루 미만,0,0
1,1세대,부부,2,대졸이상,남,68.000,혼인중,1955.000,대졸이상,비경활,1,2,NaN,아니오,두분 모두 사망 혹은 안 계심 (실종),보통,좋은 편,아니오,그런 편이다,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,예,아니오,아니오,NaN,아니오,NaN,아니오,아니오,아니오,아니오,21.470,1,5.000,과거흡연자,현재음주자,정상,0.000,0.000,0.000,0.000,4.000,0.000,0,아니오,NaN,15.000,NaN,15.000,NaN,2400.000,2415.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,전세,13000.000,30000.000,10000.000,20000.000,20.000,NaN,NaN,NaN,30.000,70.000,70.000,70.000,NaN,60.000,중의 상,아니오,하루 미만,하루 미만,하루 미만,하루 미만,5-7일,하루 미만,하루 미만,3-4일,하루 미만,하루 미만,1,1
2,1세대,부부,2,초졸이하,남,80.000,혼인중,1946.000,초졸이하,비경활,1,2,NaN,아니오,두분 모두 사망 혹은 안 계심 (실종),좋은 편,매우 좋음,아니오,그렇지 않은 편이다,예,예,아니오,아니오,아니오,아니오,아니오,아니오,예,아니오,아니오,아니오,NaN,아니오,NaN,아니오,아니오,아니오,아니오,21.970,1,7.000,과거흡연자,과거음주자,NaN,0.000,0.000,0.000,0.000,6.000,0.000,0,예,31.500,175.000,50.000,175.000,50.000,NaN,10663.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.000,10000.000,20000.000,자기집,11200.000,33000.000,10000.000,23000.000,10.000,NaN,NaN,NaN,NaN,70.000,60.000,40.000,NaN,60.000,중의 하,아니오,하루 미만,하루 이틀,하루 이틀,하루 이틀,3-4일,하루 이틀,하루 미만,3-4일,하루 이틀,하루 미만,0,0
3,1세대,부부,2,초졸이하,여,76.000,혼인중,1942.000,초졸이하,비경활,1,2,NaN,아니오,두분 모두 사망 혹은 안 계심 (실종),보통,좋은 편,아니오,그렇지 않은 편이다,예,예,아니오,아니오,아니오,아니오,아니오,아니오,예,NaN,아니오,아니오,NaN,아니오,아니오,아니오,아니오,예,예,21.480,1,5.000,비흡연자,과거음주자,NaN,0.000,0.000,0.000,0.000,6.000,0.000,0,예,19.250,175.000,50.000,175.000,50.000,NaN,823.000,100.000,50.000,5.000,0.000,0.000,13.000,15.000,5.000,0.000,0.000,10200.000,10000.000,200.000,자기집,11200.000,33000.000,10000.000,23000.000,20.000,NaN,NaN,NaN,NaN,70.000,60.000,30.000,80.000,60.000,중의 하,아니오,하루 미만,하루 이틀,하루 미만,하루 이틀,3-4일,하루 미만,하루 미만,3-4일,하루 이틀,하루 미만,0,0
4,1세대,단독가구,1,초졸이하,여,78.000,사별 또는 실종 (이산가족),NaN,NaN,NaN,1,3,NaN,아니오,두분 모두 사망 혹은 안 계심 (실종),보통,좋은 편,아니오,전혀 그렇지 않다,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,아니오,NaN,아니오,아니오,NaN,아니오,NaN,아니오,아니오,예,예,22.010,1,5.000,비흡연자,비음주자,NaN,0.000,0.000,0.000,0.000,6.000,0.000,0,예,17.000,170.000,NaN,170.000,NaN,NaN,710.000,100.000,40.000,5.000,0.000,0.000,10.000,5.000,5.000,5.000,20.000,15000.000,10000.000,5000.000,자기집,11300.000,16000.000,10000.000,6000.000,10.000,NaN,NaN,NaN,NaN,60.000,60.000,NaN,70.000,70.000,하의 상,아니오,하루 미만,하루 이틀,하루 이틀,하루 미만,3-4일,하루 이틀,3-4일,3-4일,하루 이틀,하루 이틀,0,0


#### 값 변환 후의 데이터 확인

In [60]:
rows,cols = origin2.shape
print(f"{rows} 행, {cols} 열의 {origin2.ndim} 차원 데이터")

6057 행, 101 열의 2 차원 데이터


In [61]:
# 명목형 데이터의 구성 확인 (origin2 버전)
import pandas as pd
import numpy as np
from IPython.display import display

# =========================
# 0) (선택) 출력 옵션: 생략 없이
# =========================
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# =========================
# 1) 지정한 "명목형" 컬럼 리스트
# =========================
categorical_cols = [
    'w09_fam1','w09_fam2','w09edu','w09gender1','w09marital','w09edu_s','w09ecoact_s','w09enu_type',
    'w09ba069','w09bp1','w09c152','w09c001','w09c003','w09c005',
    'w09chronic_a','w09chronic_b','w09chronic_c','w09chronic_d','w09chronic_e','w09chronic_f',
    'w09chronic_g','w09chronic_h','w09chronic_i','w09chronic_j','w09chronic_k','w09chronic_l','w09chronic_m',
    'w09c056','w09c068','w09c081','w09c082','w09c085','w09c102',
    'w09smoke','w09alc','w09addic','w09c550',
    'w09f001type','w09g031',
    'w09cadd_19','w09c142','w09c143','w09c144','w09c145','w09c146','w09c147','w09c148','w09c149','w09c150','w09c151',
    'dependent_ecotype','dependent_wage_work'
]

# (선택) 중복 제거
categorical_cols = list(dict.fromkeys(categorical_cols))

# =========================
# 2) 실제 데이터(origin2)에 없는 컬럼 자동 제거
# =========================
missing_cols = [c for c in categorical_cols if c not in origin2.columns]
if missing_cols:
    print("⚠️ origin2에 없는 명목형 컬럼(자동 제외):", missing_cols)

categorical_cols = [c for c in categorical_cols if c in origin2.columns]
print(f"✅ 최종 명목형 컬럼 수: {len(categorical_cols)}")

# =========================
# 3) dtype 표 확인 (origin2 기준)
# =========================
cat_dtype_df = origin2[categorical_cols].dtypes.to_frame('dtype').reset_index()
cat_dtype_df.columns = ['column_name', 'dtype']
display(cat_dtype_df)

# =========================
# 4) 값 구성(value_counts) 표 만들기 (origin2 기준)
#    - 텍스트로 바뀌었으니 sort_index는 문자순(가나다/알파벳) 정렬
# =========================
summary_list = []
for col in categorical_cols:
    vc = origin2[col].value_counts(dropna=False).sort_index()
    tmp = vc.reset_index()
    tmp.columns = ['value', 'count']
    tmp['column'] = col
    summary_list.append(tmp)

cat_summary_df = pd.concat(summary_list, ignore_index=True)
cat_summary_df = cat_summary_df[['column', 'value', 'count']]
display(cat_summary_df)





✅ 최종 명목형 컬럼 수: 52


,column_name,dtype
0,w09_fam1,object
1,w09_fam2,object
2,w09edu,object
3,w09gender1,object
4,w09marital,object
5,w09edu_s,object
6,w09ecoact_s,object
7,w09enu_type,object
8,w09ba069,object
9,w09bp1,object


,column,value,count
0,w09_fam1,1세대,4212
1,w09_fam1,2세대,1592
2,w09_fam1,3세대,238
3,w09_fam1,4세대,14
4,w09_fam1,기타,1
5,w09_fam2,1세대,15
6,w09_fam2,2세대,1575
7,w09_fam2,3세대,238
8,w09_fam2,4세대,14
9,w09_fam2,기타,3


### 범주형 데이터 category 타입으로 변경

In [62]:
origin3=origin2.copy()
# categorical_cols 중 실제 존재하는 컬럼만
cat_cols = [c for c in categorical_cols if c in origin3.columns]

# category로 변환
origin3[cat_cols] = origin3[cat_cols].astype('category')

In [63]:
rows,cols = origin3.shape
print(f"{rows} 행, {cols} 열의 {origin3.ndim} 차원 데이터")

6057 행, 101 열의 2 차원 데이터


In [64]:
dtype_df = pd.DataFrame(origin3.dtypes, columns=['dtype'])
dtype_df.reset_index(inplace=True)
dtype_df.columns = ['column_name', 'dtype']
pd.set_option('display.max_rows', None)

dtype_df


,column_name,dtype
0,w09_fam1,category
1,w09_fam2,category
2,w09hhsize,int64
3,w09edu,category
4,w09gender1,category
5,w09a002_age,float64
6,w09marital,category
7,w09year2,float64
8,w09edu_s,category
9,w09ecoact_s,category


- 최종적으로 각 코드들을 실제 범주 데이터로 맵핑하고
- null 값 조사 구조 기반으로 전처리 하고
- 범주형 데이터 category 형으로 변경 완료

### 추가 전처리 | 일자리에 대한 주관적 기대감 변수 4개, 1개의 컬럼으로 처리
- nan 의 경우 현재 일하지 않는 경우를 의미함

In [65]:
origin3['work_ability_age'] = (
    origin3[['w09g003', 'w09g004', 'w09g004a', 'w09g004b']]
    .bfill(axis=1)
    .iloc[:, 0]
)


#새 변수 만든 후기존 컬럼 4개는 삭제 처리
origin3.drop(columns=['w09g003','w09g004','w09g004a','w09g004b'], inplace=True)


#### 정리한 데이터 프레임 파일로 저장

In [66]:
origin3.to_csv("3.coding_book_mapping.csv", index=False)

#### 데이터 갯수 확인

In [67]:
# categorical_cols에는 있는데 origin3에는 없는 컬럼
missing = [c for c in categorical_cols if c not in origin3.columns]
print("origin3에 없는 컬럼:", missing)
print("개수:", len(missing))


origin3에 없는 컬럼: []
개수: 0


In [68]:
len(categorical_cols)

52

In [69]:
len(cat_cols)

52

In [70]:
origin3.select_dtypes(include='category').shape[1]

52